In [1]:
from model import MusicTransformer
import custom
from custom.metrics import *
from custom.criterion import SmoothCrossEntropyLoss, CustomSchedule
from custom.config import config
from data import Data
from midi_processor.processor import encode_midi, decode_midi, RANGE_VEL
from extra import *

import os
from preprocess import preprocess_midi_files_under
from progress.bar import Bar
import pickle

import utils
import datetime
import time

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter

midi_folder = os.path.join('dataset', 'midi')
preprocess_folder = os.path.join('dataset', 'preprocessed_midi')

get_config(config, ["config/thor_basic.yml"])

# load data
dataset = Data(config.pickle_dir)
print(dataset)

<class Data has "139" files>


In [10]:
dataset.all_data()

[355,
 355,
 355,
 355,
 355,
 281,
 381,
 79,
 281,
 207,
 381,
 72,
 381,
 65,
 281,
 200,
 382,
 72,
 281,
 381,
 74,
 281,
 382,
 64,
 308,
 381,
 79,
 308,
 207,
 383,
 79,
 281,
 200,
 383,
 72,
 334,
 207,
 380,
 79,
 308,
 207,
 383,
 79,
 308,
 207,
 381,
 79,
 281,
 192,
 200,
 379,
 72,
 382,
 64,
 281,
 200,
 381,
 72,
 281,
 202,
 380,
 74,
 281,
 385,
 62,
 308,
 207,
 382,
 79,
 308,
 207,
 379,
 79,
 281,
 200,
 383,
 72,
 334,
 207,
 381,
 79,
 308,
 207,
 383,
 79,
 308,
 207,
 380,
 79,
 281,
 200,
 383,
 72,
 385,
 50,
 281,
 200,
 379,
 72,
 281,
 202,
 383,
 74,
 281,
 385,
 48,
 308,
 207,
 382,
 79,
 308,
 207,
 382,
 79,
 281,
 200,
 381,
 72,
 334,
 207,
 381,
 79,
 281,
 381,
 77,
 334,
 192,
 380,
 76,
 383,
 52,
 384,
 64,
 355,
 260,
 176,
 205,
 378,
 77,
 376,
 55,
 376,
 48,
 375,
 60,
 260,
 202,
 200,
 207,
 204,
 276,
 205,
 377,
 76,
 281,
 376,
 72,
 281,
 375,
 77,
 281,
 190,
 204,
 375,
 76,
 375,
 62,
 281,
 200,
 375,
 72,
 334,
 183,
 176,
 1

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 100
learning_rate = 5e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0
# ------------
vocab_size = START_IDX['velocity'] + RANGE_VEL + 1

torch.manual_seed(1337)

data = torch.tensor(dataset.all_data())
# data = torch.tensor(dataset.batch(1,2000)[0], dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



0.257669 M parameters
step 0: train loss 6.1404, val loss 6.1605


KeyboardInterrupt: 

In [23]:
torch.save(model.state_dict(), 'models/event_predicter_8000.pth')

In [24]:
# generate from the model
context = torch.tensor([[355,
 355,
 355,
 355,
 355,
 281,
 381,
 79,
 281,
 207,
 381,
 72,
 381,
 65,
 281,
 200,
 382,
 72,
 281,
 381,
 74,
 281,
 382,
 64,
 308,
 381,
 79,
 308,
 207,
 383,
 79,
 281,
 200,
 383,
 72,
 334,
 207,
 380,
 79,
 308,
 207,
 383,
 79,
 308,
 207,
 381,
 79,
 281,
 192,
 200,
 379,
 72,
 382,
 64,
 281,
 200,
 381,
 72,
 281,
 202,
 380,
 74,
 281,
 385,
 62,
 308,
 207,
 382,
 79,
 308,
 207,
 379,
 79,
 281,
 200,
 383,
 72,
 334,
 207,
 381,
 79,
 308,
 207,
 383,
 79,
 308,
 207,
 380,
 79,
 281,
 200,
 383,
 72,
 385,
 50,
 281,
 200,
 379,
 72,
 281,
 202,
 383,
 74,
 281,
 385,
 48,
 308,
 207,
 382,
 79,
 308,
 207,
 382,
 79,
 281,
 200,
 381,
 72,
 334,
 207,
 381,
 79,
 281,
 381,
 77,
 334,
 192,
 380,
 76,
 383,
 52,
 384,
 64,
 355,
 260,
 176,
 205,
 378,
 77,
 376,
 55,
 376,
 48,
 375,
 60,
 260,
 202,
 200,
 207,
 204,
 276,
 205,
 377,
 76,
 281,
 376,
 72,
 281,
 375,
 77,
 281,
 190,
 204,
 375,
 76,
 375,
 62,
 281,
 200,
 375,
 72,
 334,
 183,
 176,
 188,
 205,
 374,
 77,
 378,
 60,
 374,
 55,
 376,
 48,
 281,
 204,
 374,
 76,
 281,
 200,
 374,
 72,
 308,
 183,
 378,
 55,
 355,
 260,
 193,
 205,
 188,
 377,
 77,
 376,
 53,
 376,
 65,
 376,
 60,
 281,
 204,
 377,
 76,
 281,
 200,
 377,
 72,
 281,
 205,
 377,
 77,
 281,
 190,
 204,
 377,
 76,
 376,
 62,
 281,
 200,
 376,
 72,
 334,
 181,
 193,
 188,
 205,
 375,
 77,
 378,
 60,
 375,
 53,
 376,
 65,
 281,
 204,
 375,
 76,
 281,
 200,
 375,
 72,
 308,
 192,
 374,
 64,
 355,
 260,
 183,
 190,
 205,
 375,
 77,
 378,
 55,
 374,
 62,
 375,
 67,
 281,
 204,
 375,
 76,
 281,
 200,
 375,
 72,
 281,
 205,
 375,
 77,
 281,
 192,
 204,
 376,
 76,
 374,
 64,
 281,
 200,
 376,
 72,
 334,
 183,
 190,
 195,
 205,
 376,
 77,
 374,
 67,
 376,
 62,
 377,
 55,
 281,
 204,
 376,
 76,
 281,
 200,
 377,
 72,
 308,
 188,
 377,
 60,
 355,
 260,
 181,
 193,
 205,
 188,
 376,
 77,
 377,
 53,
 374,
 60,
 376,
 65,
 281,
 204,
 376,
 76,
 281,
 200,
 376,
 72,
 281,
 205,
 376,
 77,
 281,
 192,
 204,
 376,
 76,
 374,
 64,
 281,
 200,
 376,
 72,
 334,
 181,
 188,
 193,
 205,
 376,
 77,
 377,
 60,
 378,
 65,
 375,
 53,
 281,
 204,
 375,
 76,
 281,
 200,
 375,
 72,
 281,
 205,
 380,
 77,
 281,
 190,
 376,
 62,
 281,
 205,
 382,
 77,
 260,
 204,
 200,
 302,
 205,
 380,
 77,
 281,
 176,
 183,
 188,
 376,
 55,
 376,
 48,
 375,
 60,
 281,
 382,
 76,
 334,
 190,
 375,
 62,
 355,
 260,
 183,
 176,
 188,
 378,
 60,
 374,
 55,
 376,
 48,
 355,
 260,
 183,
 378,
 55,
 355,
 260,
 193,
 181,
 188,
 380,
 74,
 376,
 53,
 376,
 65,
 376,
 60,
 334,
 204,
 380,
 76,
 281,
 190,
 376,
 62,
 308,
 381,
 72,
 281,
 200,
 383,
 72,
 281,
 181,
 193,
 188,
 378,
 60,
 375,
 53,
 376,
 65,
 355,
 260,
 192,
 374,
 64,
 355,
 260,
 195,
 183,
 202,
 190,
 383,
 74,
 378,
 55,
 374,
 62,
 375,
 67,
 264,
 205,
 204,
 200,
 272,
 202,
 380,
 74,
 334,
 192,
 202,
 379,
 74,
 374,
 64,
 281,
 202,
 381,
 74,
 334,
 183,
 190,
 195,
 202,
 383,
 74,
 374,
 67,
 376,
 62,
 377,
 55,
 281,
 202,
 380,
 74,
 334,
 188,
 202,
 383,
 74,
 377,
 60,
 281,
 202,
 382,
 74,
 334,
 181,
 193,
 188,
 202,
 380,
 74,
 377,
 53,
 374,
 60,
 376,
 65,
 355,
 260,
 192,
 380,
 72,
 374,
 64,
 308,
 200,
 381,
 72,
 281,
 200,
 383,
 72,
 281,
 181,
 188,
 193,
 377]])
output_sequence = m.generate(context, max_new_tokens=1000)[0].tolist()

In [25]:
decode_midi(output_sequence, file_path=f"thingithing.midi")

info removed pitch: 69
info removed pitch: 35
info removed pitch: 47
info removed pitch: 88
info removed pitch: 45
info removed pitch: 47
info removed pitch: 44
info removed pitch: 75
info removed pitch: 61
info removed pitch: 40
info removed pitch: 51
info removed pitch: 66
info removed pitch: 58
info removed pitch: 56
info removed pitch: 49
info removed pitch: 68
info removed pitch: 95
info removed pitch: 41
info removed pitch: 58


tensor([256, 381,  67, 378,  65, 379,  79, 273, 195, 193, 207, 256, 381,  67,
        378,  65, 379,  79, 273, 195, 193, 207, 256, 381,  67, 378,  65, 379,
         79, 273, 195, 193, 207, 256, 381,  67, 378,  65, 379,  79, 273, 195,
        193, 207, 256, 381,  67, 378,  65, 379,  79, 273, 195, 193, 207, 256,
        381,  67, 378,  64, 379,  79, 273, 195, 192, 207, 256, 381,  67, 378,
         64, 379,  79, 273, 195, 192, 207, 256, 381,  67, 378,  64, 379,  79,
        273, 195, 192, 207, 256, 381,  67, 378,  64, 379,  79, 273, 195, 192,
        207, 256])

In [30]:
data

tensor([16, 11,  0, 14, 22,  0, 13,  5, 21,  8,  0,  3,  0, 13, 16, 20,  9,  0,
         7, 11,  9,  9, 18,  9,  1,  0,  7, 11,  9,  9, 18,  9,  0, 12, 18,  0,
        19, 11,  9,  0,  6,  9, 18, 19,  0, 19, 11, 12, 15, 10,  0, 19, 11,  9,
        17,  9,  0,  9, 20,  9, 17,  0, 21,  5, 18,  2,  2,  0,  4,  4, 11, 11,
        11, 11, 11,  0,  7, 11,  9,  9, 18,  9])

In [12]:
list(utils.find_files_by_extensions('dataset/preprocessed_midi', ['.pickle']))
batch_x, batch_y = dataset.slide_seq2seq_batch(config.batch_size, config.max_seq)